In [1]:
import re
import pandas as pd
import numpy as np
import regex

### Python code to extract Date from chat file 

In [2]:
def startsWithDateAndTime(s):
    # pattern = '^([0-9]+)(/)([0-9]+)(/)([0-9][0-9]), ([0-9]+):([0-9][0-9]) (AM|PM) -'
    #pattern = '[0-9]{2}/[0-9]{2}/[0-9]{4}'
    pattern = '[0-9]{1}/[0-9]{2}/[0-9]{2}' # for New Group settings
    result = re.match(pattern, s)
    #print(result)
    if result:
        return True
    return False

### Regex pattern to extract username of Author.

In [3]:

def FindAuthor(s):
    # ~~~~ Produce error ~~~~~ #
    patterns = [
        '([w]+):',                        # First Name
        '([w]+[s]+[w]+):',              # First Name + Last Name
        '([w]+[s]+[w]+[s]+[w]+):',    # First Name + Middle Name + Last Name
        '([+]d{2} d{5} d{5}):',         # Mobile Number (India no.)
        '([+]d{2} d{3} d{3} d{4}):',   # Mobile Number (US no.)
        '([w]+)[u263a-U0001f999]+:',    # Name and Emoji              
    ]
    pattern = '^' + '|'.join(patterns)
    
    result = re.match(pattern, s)
    if result:
        return True
    return False

### Extracting Date, Time, Author and message from the chat file.

In [4]:

def getDataPoint(line):   
    splitLine = line.split(' - ') 
    dateTime = splitLine[0]
    date, time = dateTime.split(', ') 
    message = ' '.join(splitLine[1:])
    #if FindAuthor(message): 
    splitMessage = message.split(': ') 
    author = splitMessage[0] 
    message = ' '.join(splitMessage[1:])
    #else:
    #    author = None
    return date, time, author, message


In [5]:
### Finally creating a dataframe and storing all data inside that dataframe.
parsedData = [] # List to keep track of data so it can be used by a Pandas dataframe
### Uploading exported chat file
conversationPath = 'WhatsApp_Chat.txt' # chat file
with open(conversationPath, encoding="utf-8") as fp:
    ### Skipping first line of the file because contains information related to something about end-to-end encryption
    fp.readline()
    
    #print(fp.readline())
    #fp.readline()
    messageBuffer = [] 
    date, time, author = None, None, None
    while True:
        line = fp.readline()
        #print(line)
        if not line: 
            break
        line = line.strip() 
        if startsWithDateAndTime(line): 
            #print(100)
            if len(messageBuffer) > 0:
                #print('!!!!!!!!!!!!!!!!!')
                parsedData.append([date, time, author, ' '.join(messageBuffer)]) 
            messageBuffer.clear() 
            date, time, author, message = getDataPoint(line) 
            messageBuffer.append(message) 
        else:
            #print(200)
            messageBuffer.append(line)
    
print(len(parsedData))

df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'Author', 'Message']) # Initialising a pandas Dataframe.
### changing datatype of "Date" column.
df["Date"] = pd.to_datetime(df["Date"])

29957


In [6]:
df

,Date,Time,Author,Message
0,2021-05-11,8:33 PM,Ayesha PUCIT,Hmme na pagal ho chuki har teacher ki mintaien...
1,2021-05-11,8:33 PM,Celal PUCIT,Karain gey 👍
2,2021-05-11,8:33 PM,Ayesha PUCIT,"Kbhe sessional, kbhe assignment kbhe presenta..."
3,2021-05-11,8:33 PM,Celal PUCIT,Jitni marzi karwa lo. Bs quiz ya Viva nhi dena
4,2021-05-11,8:33 PM,Ayesha PUCIT,Or phir teachers k krny waly kam b me he kron😶
...,...,...,...,...
29952,2021-08-17,4:08 PM,Shakir Jamil,I don't need that type of aurat jo MSCS ki wja...
29953,2021-08-17,4:09 PM,Adnan Ashraf PUCIT,😂😂😂 Fir to chutti kr
29954,2021-08-17,4:09 PM,Shakir Jamil,Aho chutti hi ki hoi hai😂😂😂
29955,2021-08-17,4:09 PM,Adnan Ashraf PUCIT,😂😂😂


In [7]:
### Checking shape of dataset.
df.shape

(29957, 4)

In [8]:
### Checking basic information of dataset
#df.info()

In [9]:
### Droping Nan values from dataset
df = df.dropna()
df = df.reset_index(drop=True)

'''Don't drop, just take the rows where EPS is not NA'''

#df = df[~(df['Message'].str.len == 0)]
df = df.replace(r'^\s*$', np.nan , regex=True)
df = df.replace(r'<Media omitted>$', np.nan , regex=True)
df.shape

df = df[df['Message'].notna()]

In [10]:
### Checking no. of authors of group
df['Author'].nunique()

34

In [11]:
### Checking authors of group
df['Author'].unique()

array(['Ayesha PUCIT', 'Celal PUCIT', 'F.Bilal Uol', 'Umar Ghaffar PUCIT',
       'Amna Ali PUCIT', 'Ayesha Kiran PUCIT', 'Abdullah PUCIT',
       '+92 323 4446586', 'Khurram PUCIT', '+92 322 4155225',
       'Saad PUCIT', 'Adnan Ashraf PUCIT', 'Iqra Aslam PUCIT',
       'Shakir Jamil', 'Muflah PUCIT', 'Arshia CH PUCIT',
       'Usman CH PUCIT', '+92 304 5307578', '+92 300 4611646',
       'Muhammad Faraz PUCIT', '+92 306 2628299', 'Hafsa PUCIT',
       '+92 321 7222267', '+92 307 3900838', 'Aimen PUCIT',
       '+92 303 5018620', 'Nabeel Jutt', '+92 322 6678540',
       'Saad PUCIT MSV', '+92 335 0607000', '+92 335 5027377',
       '+92 306 4093886', 'Aneesa PUCIT', '+92 303 9326488'], dtype=object)

### Adding one more column of "Day" for better analysis, here we use datetime library which help us to do this task easily.

In [12]:
weeks = {
0 : 'Monday',
1 : 'Tuesday',
2 : 'Wednesday',
3 : 'Thrusday',
4 : 'Friday',
5 : 'Saturday',
6 : 'Sunday'
}
df['Day'] = df['Date'].dt.weekday.map(weeks)
### Rearranging the columns for better understanding
df = df[['Date','Day','Time','Author','Message']]
### Changing the datatype of column "Day".
df['Day'] = df['Day'].astype('category')
### Looking newborn dataset.
#df.head()
#df.tail()

### Counting number of letters in each message
df['Letter\'s'] = df['Message'].apply(lambda s : len(s))
### Counting number of word's in each message
df['Word\'s'] = df['Message'].apply(lambda s : len(s.split(' ')))
### Function to count number of links in dataset, it will add extra column and store information in it.
URLPATTERN = r'(https?://S+)'
df['Url_Count'] = df.Message.apply(lambda x: re.findall(URLPATTERN, x)).str.len()
links = np.sum(df.Url_Count)
### Function to count number of media in chat.
MEDIAPATTERN = r'<Media omitted>'
df['Media_Count'] = df.Message.apply(lambda x : re.findall(MEDIAPATTERN, x)).str.len()
media = np.sum(df.Media_Count)

### Looking updated dataset
df

,Date,Day,Time,Author,Message,Letter's,Word's,Url_Count,Media_Count
0,2021-05-11,Tuesday,8:33 PM,Ayesha PUCIT,Hmme na pagal ho chuki har teacher ki mintaien...,57,12,0,0
1,2021-05-11,Tuesday,8:33 PM,Celal PUCIT,Karain gey 👍,12,3,0,0
2,2021-05-11,Tuesday,8:33 PM,Ayesha PUCIT,"Kbhe sessional, kbhe assignment kbhe presenta...",51,7,0,0
3,2021-05-11,Tuesday,8:33 PM,Celal PUCIT,Jitni marzi karwa lo. Bs quiz ya Viva nhi dena,46,10,0,0
4,2021-05-11,Tuesday,8:33 PM,Ayesha PUCIT,Or phir teachers k krny waly kam b me he kron😶,46,11,0,0
...,...,...,...,...,...,...,...,...,...
29952,2021-08-17,Tuesday,4:08 PM,Shakir Jamil,I don't need that type of aurat jo MSCS ki wja...,77,17,0,0
29953,2021-08-17,Tuesday,4:09 PM,Adnan Ashraf PUCIT,😂😂😂 Fir to chutti kr,20,5,0,0
29954,2021-08-17,Tuesday,4:09 PM,Shakir Jamil,Aho chutti hi ki hoi hai😂😂😂,27,6,0,0
29955,2021-08-17,Tuesday,4:09 PM,Adnan Ashraf PUCIT,😂😂😂,3,1,0,0


In [13]:
### Extracting basic statistics from the dataset
total_messages = df.shape[0]
media_messages = df[df['Message'] == '<Media omitted>'].shape[0]
links = np.sum(df.Url_Count)
print('Group Chatting Stats : ')
print('Total Number of Messages : {}'.format(total_messages))
print('Total Number of Media Messages : {}'.format(media_messages))
print('Total Number of Links : {}'.format(links))

Group Chatting Stats : 
Total Number of Messages : 27694
Total Number of Media Messages : 0
Total Number of Links : 0


In [14]:
#Extracting basic statistics of each user : 
l = df.Author.unique()
for i in range(len(l)):
  ### Filtering out messages of particular user
  req_df = df[df["Author"] == l[i]]
  ### req_df will contain messages of only one particular user
  print(f'--> Stats of {l[i]} <-- ')
  ### shape will print number of rows which indirectly means the number of messages
  print('Total Message Sent : ', req_df.shape[0])
  ### Word_Count contains of total words in one message. Sum of all words/ Total Messages will yield words per message
  words_per_message = (np.sum(req_df['Word\'s']))/req_df.shape[0]
  w_p_m = ("%.3f" % round(words_per_message, 2))  
  print('Average Words per Message : ', w_p_m)
  ### media conists of media messages
  media = sum(req_df["Media_Count"])
  print('Total Media Message Sent : ', media)
  ### links consist of total links
  links = sum(req_df["Url_Count"])   
  print('Total Links Sent : ', links)   
  print()
  print('----------------------------------------------------------n')

--> Stats of Ayesha PUCIT <-- 
Total Message Sent :  1619
Average Words per Message :  5.920
Total Media Message Sent :  0
Total Links Sent :  0

----------------------------------------------------------n
--> Stats of Celal PUCIT <-- 
Total Message Sent :  617
Average Words per Message :  7.170
Total Media Message Sent :  0
Total Links Sent :  0

----------------------------------------------------------n
--> Stats of F.Bilal Uol <-- 
Total Message Sent :  3338
Average Words per Message :  9.190
Total Media Message Sent :  77
Total Links Sent :  0

----------------------------------------------------------n
--> Stats of Umar Ghaffar PUCIT <-- 
Total Message Sent :  1649
Average Words per Message :  7.010
Total Media Message Sent :  0
Total Links Sent :  0

----------------------------------------------------------n
--> Stats of Amna Ali PUCIT <-- 
Total Message Sent :  230
Average Words per Message :  8.260
Total Media Message Sent :  0
Total Links Sent :  0

-------------------------

In [15]:
data = [df['Message']]
rd = pd.concat(data, axis=1)

type(rd)

pandas.core.frame.DataFrame

In [16]:
rd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27694 entries, 0 to 29956
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Message  27694 non-null  object
dtypes: object(1)
memory usage: 432.7+ KB


In [17]:
### new Data Frame
#rd = df['Message']

### Remove all Emoji and other Symbols
rd = rd.replace(r'\W*$', '' , regex=True)

rd = rd.replace(r'^\s*$', np.nan , regex=True)

#rd = df[df['Message'].notna()]
rd = rd[rd['Message'].notna()]

In [18]:
rd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24457 entries, 0 to 29956
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Message  24457 non-null  object
dtypes: object(1)
memory usage: 382.1+ KB


In [19]:
rd

,Message
0,Hmme na pagal ho chuki har teacher ki mintaien...
1,Karain gey
2,"Kbhe sessional, kbhe assignment kbhe presenta..."
3,Jitni marzi karwa lo. Bs quiz ya Viva nhi dena
4,Or phir teachers k krny waly kam b me he kron
...,...
29951,😂😂😂😂 Bus Focus on your study you will get soon
29952,I don't need that type of aurat jo MSCS ki wja...
29953,😂😂😂 Fir to chutti kr
29954,Aho chutti hi ki hoi hai


In [20]:
rd = rd.sample(frac = 1)

In [21]:
len(rd)

mode = [''] * len(rd)

print(mode[:10])

rd['Mode'] = mode

### Rearranging the columns for better understanding
rd = rd[['Mode','Message']]

rd['Words'] = rd['Message'].apply(lambda s : len(s.split(' ')))

['', '', '', '', '', '', '', '', '', '']


In [22]:
rd

,Mode,Message,Words
5579,,Tu to chup kr ja desh,6
15160,,Bush kr de bush,4
9871,,Hoooo haiyeeee kia ho gya 65 lhay huay basss,10
23340,,Ya puchna ya tha k Shair kisi b traha ka send ...,15
24779,,Han g phir to any dain gy per ap k any k bgair...,17
...,...,...,...
21665,,Ye chuss hai yar,4
6681,,Nai is n aj mr jana mere haathon,8
395,,Abhi qayamat ke nobat nhe I,6
24475,,5 solid milen gy har subject me,7


In [23]:
#rd.to_csv('data.csv', index= False)  ## file size 20k+ ......

In [24]:
rd = rd[rd.Words != 1]
rd = rd[rd.Words != 2]

In [25]:
rd

,Mode,Message,Words
5579,,Tu to chup kr ja desh,6
15160,,Bush kr de bush,4
9871,,Hoooo haiyeeee kia ho gya 65 lhay huay basss,10
23340,,Ya puchna ya tha k Shair kisi b traha ka send ...,15
24779,,Han g phir to any dain gy per ap k any k bgair...,17
...,...,...,...
21665,,Ye chuss hai yar,4
6681,,Nai is n aj mr jana mere haathon,8
395,,Abhi qayamat ke nobat nhe I,6
24475,,5 solid milen gy har subject me,7


In [26]:
message = rd['Message']

In [27]:
type(message)

pandas.core.series.Series

In [28]:
for i in message:
    print (i)

Tu to chup kr ja desh
Bush kr de bush
Hoooo haiyeeee kia ho gya  65 lhay huay basss
Ya puchna ya tha k Shair kisi b traha ka send kr sakty hn na
Han g phir to any dain gy per ap k any k bgair kesy hgi attendence puri
Ik baar grade dakh ly. Phr sae sy thank you boly gy
Everyone please bs kro yr
after 2 years
Tra GB delete ni howa kia abho tak
Itny ap sary Fawad khan rehty ni
This message was deleted
ok then MOV
Mei nay kab kaha k meri kr rhay
Gustakhi ki mafi
Ni ni bna deta
This message was deleted
Saad tre baat kr ra
Yehi kisi or ny bheja hta to ap ny dher sari emojis bhejni thi
Bai thank you to hame ap ka kehna chye. Ap thank you keh k hame shirminda q kr rhi hn
tum tu barhana chahty thy teli lga k gaib hat lelyo
Hahaha wo han ha
Sahii btaon m ny khud ni dekha just marks dekhy thy
🙄🙄 baaiii me sth hun un k.. koi tng nhi kr skta unhe
Bilkul bcz ya to galt ha na  35 ma 20 22. Ya to koi baat ni bai
Chalain ruk jain gy.... wesy ma kehta hu ak week aur chlny dain yahi suspense
Me whi smjhe

Isy bare khushi ha
Ye kya bwa**er bna diye ho
Ni koi Rori ni hoi bs meri entry hoi hai
Ya b ha
😂😂😂 ha na
Ye side ly ry ya bisti kr ry
Ok ma aj e registration karvata hu en classes ma
Haha ni 2 4 logon ko e pta chli h
Sure sure me Kar dun gi
Eda tu kaka
G jb logo ka pta chal jata ky wo kaise badalty... Apky muh pe badal jayen to ap kahye ga ye
Isleyay summarizes likhwatay
This message was deleted
Bs yehi bt h na treqy sy kam hta wesy e acha ni lgta hme
Chutti kr k teri trf aa raha hu
Knjro tum se larta... mazak krta..... gussa krta... Chor jata phir wapis ajata
Change da tapik plj
Ho jata agr to m awi ye Mphill k syapy m prna tha
Ya to ha
Ye smjh nhi a raha k un ki side le rhe hen k unhe zaleel kr rhe hen
apko konse nakhre dikhaye baiiii...mre piche pre hui o
Or shukr h  mny uni jany sy phly msg dakh lia
ap tu ni bagi
bhai ap haters ko na suno...apka kaam roz 1 video bnao or group me bhjo
Hayee meri jaan
Run mureedi ki
Poch lain agr mil skta hai B- blood to
Ye mere sey ziada taiz ho gae

Ye meri glti hai ma manta hu
Bai block kiya hwa khurram ne
Wo ronay wali makhlook e nhi
Smjh jaye ga
Q jungle me rehte o kia
Ay wajji ay kaan ty😂😂😂 @923330431497 feel howa
Sakir ka msg dekha m ny soch lia ni rkhna
Mujhay ab education ko khuda hafiz keh dena chaheay
Abhi time bhi kafi ho gya hai wesy ma pta krta hu koi aisa issue na howa to btata hu tujhe
Ni filhal to ni ho ri
Abdullah ki new video a gai ha
Han ya b sae ha
Metha to ni tha kheir
Hahah mera full name yhi ta... Wo matric me hmarey aik muhtram ustad tey unho ney... Muhammad Saad kr diya
Tenu puchya kisey ne
Yr tre apni uni wale tje mu  ni lgate
Bs phir sai hai
Bhai tujh sy pahly vo mera love ha last 5 years sy
Lagta AP ny chup ka roza Rakha hota hai
Idhar to pura soap chala h
Wesy 1 meri trf sy mshwra tha k in sy b poch lain jinka nam likhy ja rhy hain ap log
@923008982661  yeh delete hoa
Shukar h koi to smjha
Copy cat wala kia scene hua yahan
Tujhe itna maru ga itna maru ga k agli 7 naslain gnji paida ho gi teri
Wohi bilku

Icm k marks bataooo sb
Abhi aik week hwa
kal report krna wese ch ka hoga zayada
Bilal ye gana bhi like kr leta hu
tujy bi pta h phir himmat bi honi chaye
you are right
Bai gussa kio krna yara tey
m phir  bi ni leave kro ga
Class honi hai abi
Tng ni krty baii khushiyan bant ty hain hm kog
Ma to soch rha tha k UOL chkr laga ao shyd vaha us ko koi kareebi e na mil jy
Tu mennu kad k vakha
Ye b acha idea hai wesy
Chalo fer jido aye gi wekhan gy
kch ziada ho ra ab bs kar do ap log
Smhjdar ho ap
Kia khyal hai
Okay tu na ana
Yay tu bohat ache bat hai...lucky hain aap log
Paper leak krne ki treat nhi hoti DC call hoti he😆😆😆 @923231488332
Bs me nai bta rha ab fr s
Bus Yarr....Just for intertainment
IT ka hisaab kuch aur e hota
bht mazakiya oo
yr m ss share tu kr doan lekin idr kuch ny bardasht ni krna k un ki koi hy ni
Mainy usy add nh kiya.. Jb me add krun ge to left nh kry ge
Sahii bt hai sir sirf page no btany aye thy k ye dekh lo yahan yahan sy prhna hai
Koi msla ni thank me later
Ma tujhe c

Bohat hi koi zalim insan ta kal koi
Yar in k male versions sy zyada khubsurat female versions hain
Ma ny nabeel sy b send karva dia
ose aggression ni kehte
Kitna fine hoga any idea
Is msg pe jwab dene lagy
iska tuk hi ni bnta
Ni ni one-on-one e sikhana ata hai mujhe bs
Isi liye to kr rhy hain
Poch kon raha hai
Tu chup kr phir mujhe rokna
Sady nal hoshyari
Bs bs hosla krien
Chal ab zyda natik mat kr
Kitna ha ap ka
Ye kya baat kr rahi ap
Han jo boys group me waha b study material share hota Shugul to yaha a kr lgate hum
Indirectly to 3no ka e msla hai
Hn naa chill the.. abhi mene f20 se suna k quizes or assignments ka scene chl raha tha un ki trf
Teri khatit to hum asi baat pa b hans lety hn jis PA banta e ni ha
😂😂🤣Bilkul b nai
Bilal me tujhy admin bna ra
Ab bus Yar
Acha tujhe koi aur kahi muh ni lgata kia jo hr waqt yaha para rehta
Phir se online
🔥🔥😂😂😂 @923054524120 ab pkka hone lga ek member km
Barriii thand hoti ja rhi🥶🥶 me jati hun ab
G blkl...GPA Ka Berra garak Kar Ra ye subject
Sho

...

ye to dangerous scene h
Amna chaaa gyi
Baki uni may be kr lain PU tou
Left Q kiya tha
Lo g ab apko meri her bt taunt lg ri group to mujhy leave krna chahiye
Mute ka option hta hai ye oper side py try krein
Jani i can understand
Tere kehney pe quiz khtm kraya
Ya aqsa kon ha ab
Happy birthday veery
Oh baii isko lrai kehty hain lrai abhi dekhi e kha h ap ny
condensed extend wala mazak na krain
Bai agr celebrities or famous personalities pe nazar ni rkhni or kis pe nazar rakhni hai
😶😶😶 te das deyo. Twano pta menu zara late smjh andi
kis type ki psnd hain tjhe
Yaad aur es ki. Asi yaad ko aag na laga du ma
Acha isko shair bhi munafqat waly hi aty hain
wo r hy
Tere se km e hain
MashaAllah last 2 hours ma 800 plus masgs. Asa kya chal rha ha aj
Acha sasty photographer
Kal. Ka. Quiz delay o skta
Mera chota jeya karnama
Paper cancel hgya tha
Emotions la kon ra.... Unj key ky tenu loose motion lagy hoye
great hygya phr to
Hafsa ko bola ha
Ty mennu kon smjhaye ga
Baii wo jis k liye lgaty jb unhein 

Haha jo jo rakhna chahta bus wahi rakhain gy na
Apki crship ma thora mitha km hota hai
Meri memory achi hai
Hahaha han yay tu h
gb whatsapp he msg late send hore  honge
Ab tu treat de dein
Sir lori de re hain
Yar abdullah ko koi kuch na kahy
koi baat ni smj a gye thi wo mistakelny send hue
@⁨Muflah UF 32⁩ and @⁨Adnan UF 37⁩  ye kam ap logo na kal kia tha to ap logo k liye😂😂😂😂😂(2
Kon konsa subject offer huwa han
Bolany ki zarurat he nh
O jao baii badshahi masjid saaf nazar a ri hai Lahore h baii
To bol ley
1- why only 4- are you komedy me
Haha adnan ki kiya karen
Hahahha.. Keh kon rha ha
Aqalmand hain hm smjh gaye hain
🤣🤣🤣Ab ye prh k mjhe v kuch yaad a rha. @923106905056? Do you
Ghor kr t b muhabbat s dooor hogyi.. Teri chorne wali aadat dekh k wo ghairat kha gya lafz pr tu nai
To us m b bs bhabi ko e dikhayen na 🙊😂 ya phr inbox e kr dia krein
ab tu manana chaye sir ko wese
Dash lanti shakal
Yakeen jaan smj ni aye
Manana? Ya manna
credit goes to me
Hahahahaha abi b to yahi kam han
Sir l

Tu menu kut pwani
Is week agar lecture ho gaya to thik ni to next
larky maza krwa diya
Ahoooo... Mera wo sheen ni hy
Me ne wo join ni kia
Tu ny mujhe admin sy hataya ha na.ab yaad rakhi. Ab ma leave kr jany laga hu. Mujhe b gussa a gya
Okay guyyssss... I have a good news
😂😂m ni chahta yr koi meri waja sy jae
Haha lokan da kala ho janda ya laal... Eda purple ho gya
Koi ni glti to ho e jati h insan hain mery sy b hui thi
Aj to ap aai thi
Oooo.. me sessionl ka scene smjh rhi thi
Yar ni mera mtlb hai k jinhon ny present kia tha sir ny unko mid final ful dye is liye 65 hain abhi baki sessionals abhi lgayen gy
Chalo dair aye drust aye. By the way Welcome
Na na na... Hun jo hona c ho gya.. cover nhi ho pae ga
ab sunu k mere sath aj kia howa... me bol rhi hn questions poch rhi hn.. magar sir ki taraf se no response... bad me pata chala k mere headphone ka mic hi band tha
Gb wtsap sy ya Fm wtsap sy
😂😂😂😂me b yhi kehny lgi thi
Yay aimen ne bataya h @923224155225
Yaar tery pas koi aur group ni ha


Kia pta sir ka dil hi na kre itna lmba msg prhne ka
Us ko me ne discuss bhi ni krna... But I am just saying.... K is group me ye any jany wala kam krne ka jis ko shoq h us ko permanently bahar ka rasta dikha dain
Ais fog ka army chief @923484608364 tu hai
Ma smjho so e raha hu
this is called chuss
Wady log h ye
Bilal dr gya
Sir tariq ny kaha hai k kal dophar tak ye assignment submit kr dirn
Ye kaha se ajata.... Kbab ki hadi
Abhi pta chala k vaccine centers mein unregistered logo ko bhi vaccine lag gai
Shakir verey to agya
Han Yar really that's amazing
To apna project bta na
ICM ka kb tk aye ga
tu sai keh raha isychuss kehty jo such keh dy
Mei nhe kru gi
Ye to sahii kha
Baith k he taliyan bjai the
Bus jb sy isko dekha hai
Get one free to hta yahan 2
Me khud ajaon ge
@923497155520 khtm tata bye bye
agar exams k marks presentation ki base pe lgy hain to presentation k marks kis ki base pe lagain gy
Phir to sahi ha class timing bta do
Coz me n search kia pr mjhe as such koi mil nai rhi.. H

Agr ye last line me clear na krte ap to sher sun kr guzra ja skta tha bnde ka
hy e.... aiwari won iz do ying parchon ke teeari
Saad ye tery liye tha isko na ignore kr
🤣🤣🤣🤣Ho e na jaye kahin
Never appears in anything and fail
🤣🤣🤣 aho tey
Boht hn jasy k Teri izat afzai vali
Mtlb baghair dekhe koi apki aankhon ka tara ban gya
Hum 2no to chalo mil k kr lety hn but ap akeli e boht ho
Ye tw zulm hai
Afsoos hai phr aisay recruiter pr
No. Ab ye facility nhi hai
Mazaak m lgy huye hain sb baii
Adi taz nazar hai teri
Class fellow ky sath hwa. Tha
Apky kitny hain muflah
Wo lfz kaha kiss na hai ye matter krta hai
Ghar ky sb se barey pe bhi bht se kaam hty
Ap k bgair ronak e ni thi yahan
Pta sbko h is scheme ka bs istemal koi koi krta hai
cr soch rai kash m dr murtaza hoti
😂 bai asan kadi serious hoye an
Slip of tounge ho gya baii...😂 actually mere kuch days se 44 hi mind pr swaar
Yeah to aqsa lg rahe hai
Mjhy b yhi offer krao
Itna bura waqt ni aya abhi
Ohh hn. Ma b karta hu dusry number pa gb aur v

In [29]:
import re
reg = '[^\sA-Za-z]'
s = 'Sir bus hamri thori izat rakh rhy hn aur methi methi kr rhy hn'
s= '😂😂😂😂😂😂😂 error to me beech me se nikal lu ga oska fiker na kro'
rs = re.findall(reg,s)

In [30]:
len(rs)

7

In [31]:
def remove_emoji_msg(msg):
    import re
    reg_Exp = '[^\sA-Za-z]'
    len(msg)
    new_msg = []
    for i in msg:
        rs = re.findall(reg_Exp, i)
        if len(rs) == 0:
            new_msg.append(i)
    return new_msg

In [32]:
_list = remove_emoji_msg(message)

In [33]:
_list[:10]

['Tu to chup kr ja desh',
 'Bush kr de bush',
 'Ya puchna ya tha k Shair kisi b traha ka send kr sakty hn na',
 'Han g phir to any dain gy per ap k any k bgair kesy hgi attendence puri',
 'Everyone please bs kro yr',
 'Tra GB delete ni howa kia abho tak',
 'Itny ap sary Fawad khan rehty ni',
 'This message was deleted',
 'ok then MOV',
 'Mei nay kab kaha k meri kr rhay']

In [34]:
len(_list)

14282

In [35]:
print(type(rd))

for index, row in rd.iterrows():
    print (row["Message"], row["Words"])

<class 'pandas.core.frame.DataFrame'>
Tu to chup kr ja desh 6
Bush kr de bush 4
Hoooo haiyeeee kia ho gya  65 lhay huay basss 10
Ya puchna ya tha k Shair kisi b traha ka send kr sakty hn na 15
Han g phir to any dain gy per ap k any k bgair kesy hgi attendence puri 17
Ik baar grade dakh ly. Phr sae sy thank you boly gy 12
Everyone please bs kro yr 5
after 2 years 3
Tra GB delete ni howa kia abho tak 8
Itny ap sary Fawad khan rehty ni 7
This message was deleted 4
ok then MOV 3
Mei nay kab kaha k meri kr rhay 8
Gustakhi ki mafi 3
Ni ni bna deta 4
This message was deleted 4
Saad tre baat kr ra 5
Yehi kisi or ny bheja hta to ap ny dher sari emojis bhejni thi 14
Bai thank you to hame ap ka kehna chye. Ap thank you keh k hame shirminda q kr rhi hn 20
tum tu barhana chahty thy teli lga k gaib hat lelyo 11
Hahaha wo han ha 4
Sahii btaon m ny khud ni dekha just marks dekhy thy 11
🙄🙄 baaiii me sth hun un k.. koi tng nhi kr skta unhe 13
Bilkul bcz ya to galt ha na  35 ma 20 22. Ya to koi baat ni b

🙏🏻🙏🏻 Ap sy zyda nii 5
yr ab ye kia dramey bazi e 7
remaining presentations nhi ly rhy wo 6
Aasma leave it Yar 4
Meri traf sy v 4
Bus avy e. A jy ga again 7
Yani sb parhako ho aur yahan sb dramay krtay ho 10
Vo q karna ha 4
Haha khuram bechary ko koi serious ni leta 8
aesa hi he....me to chahta hu poori degree online o jaye 11
ja kam kr apna 4
m shair sunao tu aag lga doan lekin llrkiyan mujy ghalt smjhien gy q k  sirf hafi k shair parta hoan 22
Power sb sy pehly 4
Yr 2-4 din ka time lo. Ye log pagal h kya 11
Allah Allah maaf kr dein mujhy 6
Ha ye to hai 4
lar da q aa 4
Bai uski baat ka jwab diya ta 7
Ma to class ma hu e ni 7
Apka female version dekh k ab seekha dain gy 9
Baii ayesha kiran na itny din bad msg kia hansan to bnta tha na 14
To phir ab kia change hona 6
hahahah awain gusa chrahta rehta 5
Tuj ma wo bat ni jo bilal ma hai 9
Ap ka b toota hua 5
Bol kon rha ha 4
@923330431497 ku sara gusaa apni trf divert kr rae ho 10
Same....big no to CV....wo b ek female teacher se 9
Summary:

Yaar ya jo b huva ha huva galat ha vase. Ab class 2 hiso ma bant gai ha 18
Vibes to esi e a ri hain 7
Hahaha ab bechara ro rha ha baith k aur hame b Rulla rha ha 14
Na baii hm sy 1 kam ni ho ra sahii trah 11
Hahhahaha ye b sahi ha 5
Haha ye b sahii h 5
Bai adnan to brand ha 5
🙏🙏 bahi meharbani kr 4
Meri position btayie ga plz 5
Koi ni eda zarori v ni si 7
Abhi sor k kehnay ka wait krain phir 8
Ha thak gaye hain 4 din ki thakan hai 9
😂😂😂😂 ab thk he phr 5
Aisi lela hi nhi chahiye 5
Kari v na 3
B**k. Yr ghr beth beth bor ho gye 8
Ahooo... Is ky sahi mazy hain 6
Wesay herat h aik week off de deya inonay ne ....shukar h thora sakoon karein gy ab 17
Shakir or iqra kahan gy 5
Description updated of our group 5
Assalam-U-Alikum everyone.. Today is the last date (25th May) for the fee submission of 2nd semester for MSCS 18
chalo saad khair jo h stay blessed 7
Aur mujhe to khushi hi khushi hoi 7
Then apko koi tention ni honi chaiye 7
Wohee kr rhee hu abh 5
Hmain tw sb sy pechy khara kr daity thy

Iqra bhia hoti hai 4
Hm bhi kr dain phir mail 6
me typing chorr k idhr bat krny lag gae thi 10
Es pa Hansna tha 4
Kia hgya baii 3
yes sir i am following u regularly 7
Yep. Jaan h apni. Hum hi lga sakty shugal bki kise horda bardsht ni hnda 15
Raat ko vibes ai thi thori per ab zra serious ly jaty mazak ko 14
Dakho bilal k khilaf ma ak lfz ni sun skta 10
Hahahhha han sub active rehty 5
Han jani ye to sahii kha apni khi hui baton sy e mukar jaty hain 15
Thori dair bad try krna 5
@923054524120 Bhai Mahzrat 3
Welcome back cr shaiba 4
inko inbox kr do heart bna k 7
Serious e ho gaye ap tw 6
Yarrr guide tb krta h jb experience ho... Ab hm guide kr sakty hain apko 15
Msla koi ni 3
Ni ghr agya 3
Hun family group ni ay 5
Abi bohat halka rkha e Ye konsa lrki e jisko jwab dene se pehle me ne sochna.... Ye jitni taiz ball phenke ga utna lmba chakka jana 27
Enu kon entry de denda etho di 7
😆😆🤣🤣 kr dia maaf 4
Hona b ni chahta 4
Chalo yr miti pao 4
apke sessional kitne hain Arshia 5
Hahaha o waki e 4


Ye heavy tha 3
Aisa kia ho gya 4
Dusri side pa a vaha batata hu 7
Wese @923497155520 hen kahan?? Nzr nhi a rhe 8
Mery khyl ma to ya pre planned tha 8
Ha na logo ka to kam e ye hai so forget them 12
Suchi mjy nh pata.. Koi bta kiya h ye 9
Hansna tha kia 3
Zuban phisal gai 3
Esa to na khein yar 5
upload q ni ki 4
Koi idea ni 3
bachy ki jan lo gy kya 6
Phir me IOT or data mining me se aik chor saku gi 12
Itna upset na hwa krien 5
Lagda menu GB install krni paini 6
Han but hum log to again b gy thy to es lie pta ha 14
mje tra status q ni show ota 7
Mu se supari nikal k bol Aese smj ni a rae 11
Keh kon ra hay 4
Apko behtar pta hona chahiye 5
Mtlb koi umeed lgai ja skti h 7
Hahahaha.. Tm khien or he nikal jaya kro 8
Ni ni aisa na krain 5
Apke kehne ka mtlb h jo ye kehta wo pagal hta 11
A jaye gi awaz itni b bt ni h ab 10
Ap shukr karo, apne 2 most difficult courses online mein pass kr liay. ACA & ToC. Aagay ab aasani hai 20
Video dakh k aur kia positive sochty 7
Q tu ka ******* ha 5
Ni tm lo

...

This message was deleted 4
Main Kia keh sakta hn 5
Meri baby ne eid snaps bnai 6
Ak to yaha k admin bachy hain 7
Ye kia bat hai 4
mera thanks na bolien in ka bi sukriya bol dein itni khoshis kr rahe hain 15
Bro 2 din ki CR h brdasht kr lyn 😂🙉🙊  Ups sorry 13
Iss se kuch nhi chupa 5
Ye aur munna kakka 4
Awain filmein chla ra ta ye 6
m tanu sariyan nu block kr reya 7
@923224155225 sai keh rai hain 5
Sb kuch ap hi hain 5
Bhai inko stage p moka nahi mila warna bht mashoor hna tha bhai ne 14
Ni ye sb asal me khud ko tasalli de rhy thy 11
Agr phr b na saad na maane to SS dikha dena 11
Isi liye to video bheji h unhon ny 8
G apko bri achi trh pta ho ga 8
confidence boht hy bai 4
Or unity ki bat kr rhy thy 7
Shugal mela lganey waliyan 4
Even mere pe barsaaat kr deni gussay ki aasma n 10
Exactly.. Takey hm mazeed chill krein 6
Koi ghor e ni kr ra 6
Nh bht kam ha 4
Ni ye ni kru ga save 6
Ye kia chuss hai 4
40 hn mid aur final k mila k 8
blkl freeeeeee.......ly lo😁😁bila The Great se 6
Han Han ab ap

*Sir Ather has announced the failed students' name in the group. It means all other will pass the subject 19
Matlb na jao na phr 5
Han mery lehaz sy to 5
Ma kasy band kro bai. Mery pas to chabi e ni ha 12
yh ap pehly bi kr sakhti ti 7
Tu bhai b ni manta... Bhjo SS 7
Nabeel bandy da putar ban ja 6
Oooo bai bai bai 4
Apko to faida tha naw 5
Ha wo to jaye ga hi jaye ga 8
This message was deleted 4
Apna scene clear krien 4
Kb kaha ha us ny. Masg dekha phr. Us ny kaha ha k us ko chir ha ab en sy 20
G bilkul ...nazar araha hai 5
Sir ko to awaz ni jani 6
G... Ye group ap logo ka hai 7
Me CMS pe dekh ra ta 6
aj khair h mosam acha h tu log wazriamosmiyat ly aaye 11
Natik baaz🤣🤣😂 drama queen 4
Hahaha. Bary level k din thy ya b 8
Koi assaan trakia ni ha? Kya 6
Ye group ka name q aesa rkh dia 8
Matlab ma b senior ban gya 6
To abhe tak formal ho rhe the 7
apko b kirush a gia e 6
Na bai na... Larko ka din subha shuru hta 9
😂 bilkul nhi suit kiyaa 5
Lga len aag ap 4
Whi to... 😐. Koi to smjhe ga kabhi

Sir super admin ka tag mere se belong nahi karta 10
Repeat krne ka irada e 5
Chup kr ja. Fir tu boly ga mujhe q nikala 10
Keh kon ra hay 4
face app se female ka male version b bn jata e 11
Isi liye bta k gai hain aj wo 8
Just u r important budz 5
Hahahaha oh acha solili 4
Sai e sai kri jao shaper triple chruble.... Hun ni me krda koi v gal 15
لازم نہیں حیات میں احباب کا ہجوم ہو پیکر خلُوص تو کافی ہے ایک شخص 15
yr  m wese sona ni lg raha 8
Knse store s ye wali pill milti 7
yeh nai pta 3
Meri taangein torne ka kaha 😭😭😭. Soch b kaisay lia 10
Han waqie ke to kab s keh rha tjhe k mar ja tapak ja 😂😂 Aj pata chala me kiun kehta tha 22
Tiktok star hai ye 4
Chalu 15 or 17 closest to thy na 8
Han ho b sakta ha 5
m ny liver kb kaha tujy 6
Thank you Aasma. U r doing a fantastic job 9
Hahaha sae sae sae sae sae sae sae 8
Oye quiz ki tiyari kr ly raat ko mujhe parhana hai tu na 13
Second semester me ye first subject jis k liye me confident ho rhi hn k ye parhna h 🤩 warna baki subjects k bary me mera

Adnan to to luv h biro 6
Hm n apna kaam kia to fr v nai rehna tu ithay 12
Now i have no stickers 5
Aho Shakir b sath e hn 6
Allah muaf krey. sab se khamoosh insan ho me 9
Yr choro jany do chill kro 6
patience ki kami 3
admin hoga tu ghr m 5
Masg pa goor kro. Spelling ko goli maro 8
bahi m ny tu abi dekh liya h procedure 9
Ye check kr aimen 4
jinhoan ny proj ni submit krwaya un k sessional zero 10
Ha Allah khai e kry bs 6
Yar paper to bad main bhe ho jata hai...pr yeah kam 11
Aoa aaj DMIR ka makeup  lecture and quiz hai 9 bjay k nhe 14
That's is the beauty of chat 6
Han bs marks dakh kr bag m dal dia tha or us k bad abi tk nai dakha dobara 19
Ghalti kr di yaaar tu ne 6
Bai tu msg krey or me reply na dun 9
Ye respect or pucit do opposites kabhi aik sath nahi ho sakty 12
Awen awen...Mene to kch b ni kaha😅😅khud e bol Rae 10
Eyh v gal higi 4
Lgta hai adnan ki new video any wali hai phir 10
Han mujhe idea ho rha ha 6
Bilal ghalati se aj chala gaya 6
Ghalt nai asal wala aya 5
Kuch aur kam ni 

Wo khush ho rai thi 5
Ap ko covid huva tha 5
😂😂😂😂bat to sach hy magr bat hy 7
Hun bolo CR + Admin is wada than super admin 10
Yar bs nai umeed to chill krein yahan wesy b kisi ko koi frq ni prta to jany dain 19
Ye b nh pata apko.. @923484608364 Ye dost kehta ha apko 11
Kaha gaib ha aj tu subha sy 7
Wah bai. Good ho gya phr to 7
aj k bad Aasma ko tang ni krna wo boht achi hain 12
Bii mujhe ni pta aysha ka sachi yar 8
Yaar ye ksm se hmara ak doat krta tha uss na Humera Malik k name se ID bnai thi 19
chalo sarey wekho 3
Yup.. mje koi issue nhi kisi ki satisfaction se... Me just apne point pr baat kr rhi thi 18
Iska matlb hai bhai sb dil ky toty ho ap 10
Kuch khas ni 3
Esky siwa ek b smhj nh aya 7
Nu yar uska mza ni tha 6
Hayee ye to gehri bt kr di yaar 8
Saad ny likha to huva tha Important discussion 8
kya yr nonprofessional behaviour from fellows 6
Aisy kesy chor du badi mushkil se to hath laga hai 11
Umar bta skta hau ye to 6
Smjhao umer yar iqra ko smjhao 6
ijszat ni hy bai 4
Jitni grm

In [37]:
len(rd)

20397

In [46]:
new_df = pd.DataFrame(_list, columns=['Message'])

In [47]:
mode = [''] * len(new_df)

print(mode[:10])

new_df['Mode'] = mode

### Rearranging the columns for better understanding
new_df = new_df[['Mode','Message']]

['', '', '', '', '', '', '', '', '', '']


In [48]:
new_df

,Mode,Message
0,,Tu to chup kr ja desh
1,,Bush kr de bush
2,,Ya puchna ya tha k Shair kisi b traha ka send ...
3,,Han g phir to any dain gy per ap k any k bgair...
4,,Everyone please bs kro yr
...,...,...
14277,,Bhai goli v chaye ya hwai fire marne
14278,,Ye chuss hai yar
14279,,Nai is n aj mr jana mere haathon
14280,,Abhi qayamat ke nobat nhe I


In [49]:
# Export dataframe to CSV
new_df.to_csv('chat_data.csv', index=False)